In [2]:
import numpy as np

##### Constants (Peacemaker)

In [3]:
pi = 4.0 * np.arctan(1.0)
planck = 6.62606957e-34         # J s
avogadro = 6.0221413e23
kb = 1.3806488e-23              # J K^-1
speed_of_light = 299792458.0    # m s^-1
amu = 1.660538921e-27           # kg
gas_constant = avogadro*kb
hbar = planck/(2.0*pi)
global_eps = 1.0e-10

# Partition Functions

### Translational Partition Function (calculate_lnqtrans)

$q_{\text{trans}} = \frac{V-b_{xv}V_{\text{excl}}}{\Lambda^3}$ </p>
$\Lambda = \sqrt{\frac{h^2}{2\pi mkT}}$

In [4]:
# Calculates the partition function for one cluster

def calculate_lnqtrans(lnqtrans, bxv, temp, v, mass, v_excl):
    # lnqtrans  : input/output  - ln of the translational partition function
    # bxv       : input         - van der Waals scaling factor 
    # temp      : input         - temperature
    # v         : input         - volume of the cluster
    # mass      : input         - mass of the cluster
    # v_excl    : input         - excluded volume
    
    m = mass*amu
    lambda_ = planck/np.sqrt(2.0*pi*m*kb*temp)
    
    lnqtrans = np.log(v-bxv*v_excl) - 3.0*np.log(lambda_)
    return lnqtrans

#### Tests

In [5]:
lnqtrans = 0
bxv = 0.8
temp = 298.0
v = 2.0
mass = 0.001
v_excl = 0.1
lnqtrans = calculate_lnqtrans(lnqtrans, bxv, temp, v, mass, v_excl)
print(lnqtrans)

59.33445379941631


### Vibrational Partition Fuction (calculate_lnqvib)

#### Harmonic oscillator
$q_{\text{vib}} = \frac{\exp(-\Theta^{\text{vib}}/2T)}{1-\exp(-\Theta^{\text{vib}}/T)}$ </p>
$\Theta^{\text{vib}} = \frac{h\nu}{k_B}$ </p>
$\frac{h\nu}{k_BT} = \frac{hc}{k_BT} \cdot \~{v}$ 

#### Morse oscillator (Anharmonic effects in the quantum cluster equilibrium method -2017) 
$E_n = hc\nu [(n+ \frac{1}{2})-\mathcal{X}_e(n+ \frac{1}{2})^2]~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ 
$\mathcal{X}_e$ : the anharmonicity constant </p>
$q_i^{\text{morse}} = \prod_{\mu=1}^{\mu_{\text{max}}} \sum_{n=0}^{n^{\text{max}}} \exp(-(n+ \frac{1}{2})) \frac{\Theta_{i,\mu}^{\text{vib}}}{T}+\mathcal{X}_e(n+ \frac{1}{2})^2\frac{\Theta_{i,\mu}^{\text{vib}}}{T}~~~~~~$  $\mu$ : vibrational modes </p> 
$~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ 
$n^{\text{max}}$ : highest vibrational energy level before dissociation for the respective mode </p>

$\Rightarrow$ Strekalov $\Rightarrow$ </p>
$q_i^{\text{morse}} = \prod_{\mu=1}^{\mu_{\text{max}}} \frac{1}{2 \cdot \sinh(\frac{\Theta_{i,\mu}^{\text{vib}}}{2T})} \exp(\mathcal{X}\frac{\Theta_{i,\mu}^{\text{vib}}}{T}(\frac{1}{4}+\frac{1}{2 \cdot \sinh(\frac{\Theta_{\mu}^{\text{vib}}}{2T})^2}))$ </p>

#### Free rotator (Supramolecular Binding Thermodynamics by Dispersion-Corrected Density Functional Theory - Grimme 2012)

$\mu = \frac{h}{8 \pi^2 \nu}$ </p>
$\mu' = \frac{\mu B_{\text{av}}}{\mu + B_{\text{av}}}~~~~~~~~~~~~~~~~~~~~~~~$ Moment of inertia of the cluster with frequency $\omega$</p>
$B_{\text{av}} = \sum_{n} \frac{I_n}{N}~~~~~~~~~~~~~~~~~~~$ Avarage moment of inertia of the cluster</p>
$T_{\text{rot}} = \frac{\hbar^2}{2 k_{\text{B}} \mu'}$ </p>
$q^{\text{rot}} = \frac{1}{\sigma} \sqrt{\frac{\pi T}{T_{\text{rot}}} }~~~~~~~~~~~~~~~~~$ $\sigma$ rotational symmetry nuber </p>
$w(\omega) = \frac{1}{1+(\text{rotor cutoff}/\nu)^4}$

In [6]:
# Calculates the vibrational partition function for one cluster

def calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const):
    # lnqvib       : input/output  - ln of the vibrational partition function
    # temp         : input         - temperature
    # freqs        : input         - vibrational frequencies
    # inertia      : input         - moments of inertia (number of moments of inertia depends on shape of the cluster)
    # sigma        : input         - symmetry number of the cluster
    # rotor_cutoff : input         - cutoff frequency for the free rotor approximation
    # anharm_const : input         - anharmonicity constant

    # hc/k
    factor = planck * speed_of_light * 100 / kb 
    # Avarage moment of inertia
    Bav = sum(inertia)/len(inertia) * amu * 1.0e-20

    # Initialize variables
    lnq_aho = 0.0
    lnq_ho = 0.0
    lnq_fr = 0.0
    lnqvib = 0.0
    w = 1.0
    
    for i in range(len(freqs)):
        # Case 1: Harmonic oscillator
        if (anharm_const == 0.0):
            t_vib = factor * freqs[i]
            lnq_ho = -t_vib / (2.0 * temp) - np.log(1 - np.exp(-t_vib / temp))

        
            # Case 1.1: Free rotor
            if (rotor_cutoff > 0.0):
                mu = planck/(8.0 * pi**2 * freqs[i] * speed_of_light * 100.0)
                mu_prime = mu*Bav/(mu+Bav)
                trot = hbar**2/(2.0*kb*mu_prime)
                lnq_fr = np.log(1.0/sigma * (np.sqrt(pi*temp/trot)))
                w = 1.0/(1+(rotor_cutoff/freqs[i])**4)
                
        # Case 2: Anharmonic oscillator
        else:
            t_vib = factor * freqs[i]
            # (Logarithm laws)
            lnqvib = lnqvib - np.log(2.0 * np.sinh(t_vib / (2.0 * temp))) + anharm_const * (t_vib / temp) * (0.25 + 1.0 / (2.0 * np.sinh(t_vib / (2.0 * temp))**2))
        
        lnqvib += w * lnq_ho + (1.0 - w) * lnq_fr
    return lnqvib

##### Tests

In [7]:
# Case 1 : Harmonic oscillator
lnqvib = 0.0
temp = 315.00
freqs = [11.4, 25.6, 68.7, 134.9, 3555.9]
inertia = [212.4, 212.4]
sigma = 2
rotor_cutoff = 0
anharm_const = 0.0
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

result -1.3967910870430646


In [8]:
# Case 1.1 : Harmonic oscillator with free rotator
lnqvib = 0.0
temp = 298.00
freqs = [5.0, 648.0, 1000.0, 3555.7]
inertia = [3.2, 5.9, 89.3, 1.0]
sigma = 1
rotor_cutoff = 3000.0
anharm_const = 0.0
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

# Case 1.1 : Harmonic oscillator with free rotator
lnqvib = 0.0
temp = 298.00
freqs = [100.0, 200.0, 300.0]
inertia = [1.0, 1.0, 1.0]
sigma = 1
rotor_cutoff = 3000.0
anharm_const = 0.0
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

result -3.855446885285766
result 1.7676272441463634


In [9]:
# Case 2 : Anharmonic oscillator
lnqvib = 0.0
temp = 298.00
freqs = [180.0, 270.0, 3550.0]
inertia = [1.0, 3.0, 1.0]
sigma = 1
rotor_cutoff = 3000.0
anharm_const = 0.6
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

# Case 2 : Anharmonic oscillator
lnqvib = 0.0
temp = 315.00
freqs = [5.0, 648.0, 1000.0, 3555.7]
inertia = [38.6, 43.9, 112.9]
sigma = 1
rotor_cutoff = 0
anharm_const = 0.34
lnqvib = calculate_lnqvib(lnqvib, temp, freqs, inertia, sigma, rotor_cutoff, anharm_const)
print('result',lnqvib)

result -3.8012813311299443
result 23.9041034243484


### Rotational Partition Function (calculate_lnqrot)

$q_{\text{rot}} = 1~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~$ Atom </p>
$q_{\text{rot}} = \frac{1}{\sigma}  \cdot \frac{T}{\Theta^{\text{rot}}}~~~~~~~~~~~~~~~~~~~~$ Linear cluster </p>
$q_{\text{rot}} = \frac{1}{\sigma}  \cdot \sqrt{\frac{\pi T^3}{\Theta^{\text{rot}}_1\Theta^{\text{rot}}_2\Theta^{\text{rot}}_3}}~~~~~$ Nonlinear cluster </p>
$T_{\text{rot}} = \Theta^{\text{rot}} = \frac{\hbar^2}{2 k_{\text{B}} I}$ </p>

In [10]:
# Calculates the rotational partition function for one cluster

def calculate_lnrot(lnq, temp, inertia, sigma, atom, linear):
    # lnq     : input/output  - ln of the rotational partition function
    # temp    : input         - temperature
    # inertia : input         - moments of inertia (number of moments of inertia depends on shape of the cluster)
    # sigma   : input         - symmetry number of the cluster
    # atom    : input         - boolean, true if cluster is atom
    # linear  : input         - boolean, true if cluster is linear
    
    
    # Case 1: Cluster is an atom
    if (atom):
        lnq = 0.0
    # Case 2: Cluster is not an atom
    else:
        lnq = 1.0
        for i in range(len(inertia)):
            # Calculate the rotational temperature
            trot = hbar**2/(2.0*kb*inertia[i]*amu*1.0e-20)
            # Calculate the part of the rotational partition function 
            # that is equal for linear and non-linear clusters
            lnq *= temp/trot
            
        # Case 2.1: Linear cluster
        # The inertia array contains two equal moments of inertia
        # The square root is taken to account for the degeneracy of the rotational levels
        if (linear):
            lnq = 1/sigma * np.sqrt(lnq)
            
        # Case 2.2: Non-linear cluster
        # The inertia array contains three moments of inertia
        else:
            lnq = 1/sigma * np.sqrt(pi*lnq)
            #print('lnq n',lnq)
                
        lnq = np.log(lnq)
            
    return lnq
            

#### Tests

In [11]:
# Case 1: Cluster is an atom
lnq = 0.0
temp = 298.00
inertia = [1.0, 1.0, 1.0]
sigma = 1
atom = True
linear = False
lnq = calculate_lnrot(lnq, temp, inertia, sigma, atom, linear)
print('result',lnq)

result 0.0


In [12]:
# Case 2.1: Linear cluster
lnq = 0.0
temp = 298.00
inertia = [37.4, 37.4] 
sigma = 1
atom = False
linear = True
lnq = calculate_lnrot(lnq, temp, inertia, sigma, atom, linear)
print('result',lnq)

result 6.130167446075413


In [13]:
# Case 2.2: Non-linear cluster
lnq = 0.0
temp = 298.00
inertia = [21.2, 43.2, 70.0]
sigma = 2
atom = False
linear = False
lnq = calculate_lnrot(lnq, temp, inertia, sigma, atom, linear)
print('result',lnq)

result 9.17613133433584


### Electronic Partition Function (calculate_lnqelec)

$q_{\text{elec}} = g \cdot \exp(-\frac{E_{\text{elec}}}{kT})$ </p>

In [14]:
# Calculates the electronic partition function for one cluster

def calculate_lnqelec(lnqelec, temp, energy):
    # lnq  : input/output  - ln of the electronic partition function
    # temp : input         - temperature
    
    lnqelec = -1.0/avogadro * energy/(kb*temp) * 1000.0 # kJ/mol to J/mol (energy is in kJ/mol)
    return lnqelec

#### Tests

In [15]:
lnqelec = 0.0
temp = 318.15
energy = -28.18
lnqelec = calculate_lnqelec(lnqelec, temp, energy)
print('result',lnqelec)

result 10.653072928364542


### Mean Field partition function (calculate_lnqint)

$ \epsilon_{\text{mf}} = - a_{mf} \cdot \frac{N}{V}$ </p>
$ N = \text{number of components} \cdot  N_{\text{tot}}$ </p>
$ N_{\text{tot}} = \sum^{N_{\text{components}}} \text{monomer amount of the component (\%)} \cdot N_\text{A} = N_\text{A}$ </p>
$ ln(q_{\text{int}}) = -\frac{\epsilon_{\text{mf}}}{K_{\text{B}}T} $ </p>

In [16]:
# Calculates the mean field partition function for one cluster

def calculate_lnqint(lnqint, amf, temp, v, composition, ntot):
    # lnqint       : input/output  - ln of the mean field partition function
    # amf          : input         - van der Waals pressure scaling factor
    # temp         : input         - temperature
    # v            : input         - volume of the cluster
    # composition  : input         - composition 
    # ntot         : input         - total number of particles
    
    e_mf = -amf * sum(composition) * ntot / v
    lnqint = -e_mf / (kb * temp)    
    
    return lnqint

#### Tests

In [19]:
lnqint = 0.0
amf = 1.0e-48
temp = 450.0
v = 3.5e-3
composition = [1, 6, 2]
ntot = (0.3 + 0.5 + 0.2) * avogadro
lnqint = calculate_lnqint(lnqint, amf, temp, v, composition, ntot)
print('result',lnqint)

result 0.2492468468447588
